# Using promptBench to evaluate LLama 2 Chat models

***

PromptBench is a unified library for evaluating and understanding large language models.

With promptBench, we can:
- **Quickly access your model performance**: PromptBench provides a user-friendly interface for quick build models, load dataset, and evaluate model performance.
- **Prompt Engineering**
- **Evaluate adversarial prompts**: PromptBench integreates prompt attacks: for researchers to stimulate black-bock adversarial prompt attacks on the models and evaluate their performances.


---
This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|llama-2-chat-completion.ipynb)

---

---
In this demo notebook, we will demonstrate how to use promptBench, the unified library to evaluate and understand large language models. We will be using a llama 2 model on Amazon Bedrock by creating a class that integrates Meta's Llama 2 Chat models on Amazon Bedrock.

We will begin by installing promptbench via the pip command. Once we have promptbench installed, we can change to the promptbench directory that is already in this lab and install the required packages in the requirements.txt file.

---

## Install promptBench package dependencies

To begin benchmarking with promptbench, we begin by installing all of the promptbench library dependencies and installing the requirements.

This example will walkthrough the basic usage of PromptBench and being able to leverage the promptbench library.

PromptBench supports various datasets for the evaluation of large language models (LLMs) and is designed to be a unified library for evaluating and understanding LLMs. It consists of several key components that are easily used and extensible for reseachers, including prompt instruction, prompt engineering, dataset and model loading, adversarial prompt attack, dynamic evaluation protocols, and analysis tools.

In [1]:
pip install promptbench

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -r requirements.txt

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
promptbench 0.0.2 requires transformers==4.35.2, but you have transformers 4.36.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Promptbench / Amazon Bedrock dependencies 

Let's begin by importing promptbench and bedrock dependencies for our environment. This will allow us to invoke our llama 2 model on Amazon Bedrock and evaluate the accuracy of the sentiment analysis prompts that are loaded through the 'sst2' dataset.

In [119]:
import promptbench as pb
import bedrock
import botocore
import json
import os

In [120]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [136]:
# used to understand model syntax, can skip this cell.
help(pb.models.LMMBaseModel)

Help on class LMMBaseModel in module promptbench.models.models:

class LMMBaseModel(abc.ABC)
 |  LMMBaseModel(model_name, max_new_tokens, temperature, device='auto')
 |  
 |  Abstract base class for language model interfaces.
 |  
 |  This class provides a common interface for various language models and includes methods for prediction.
 |  
 |  Parameters:
 |  -----------
 |  model : str
 |      The name of the language model.
 |  max_new_tokens : int
 |      The maximum number of new tokens to be generated.
 |  temperature : float
 |      The temperature for text generation (default is 0).
 |  device: str
 |      The device to use for inference (default is 'auto').
 |  
 |  Methods:
 |  --------
 |  predict(input_text, **kwargs)
 |      Generates a prediction based on the input text.
 |  __call__(input_text, **kwargs)
 |      Shortcut for predict method.
 |  
 |  Method resolution order:
 |      LMMBaseModel
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  


## Implement llama 2 on Amazon Bedrock interface

Now we will create a Llama2Bedrock class that will allow you to invoke your llama 2 model by acting as the model interface for llama 2 on Amazon Bedrock.

The class will read the following parameters:
- 'modelID:' the ID of the Llama 2 model.
- 'max_gen_len': the maximum number of new tokens to be generated.
- 'temperature': the temperature for text generation (default is 0.2).
- 'top_p': iff set to a float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation (default is 0.9).

The Llama2Bedrock class will be used to load the llama 13b chat model and generate text based on the prompt input.

The **'__init__'** method will initialize the model with the provided parameters while the **'predict'** method will generate text based on the text input and the specified model parameters. 

In [123]:
class Llama2Bedrock(pb.models.LMMBaseModel):
    """
    Language model class for interfacing with Llama2 models on Amazon Bedrock.

    Inherits from LMMBaseModel and sets up a model interface for Llama2 models on Amazon Bedrock.

    Parameters:
    -----------
    modelId : str
        The Id of the Llama2 model.
    max_gen_len : int
        The maximum number of new tokens to be generated.
    temperature : float, optional
        The temperature for text generation (default is 0.2).
    top_p : str, optional
        If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation. (default is 0.9).
    """
    def __init__(self, modelId, bedrock_runtime, system_prompt=None, max_gen_len=1024, temperature=0.2, top_p=0.9):
        super(Llama2Bedrock, self).__init__(modelId, max_gen_len, temperature, top_p)
        self.modelId = modelId
        self.bedrock_runtime = bedrock_runtime
        self.max_gen_len = max_gen_len
        self.top_p = top_p
        if system_prompt is None:
            self.system_prompt = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
        else:
            self.system_prompt = system_prompt
    
    def predict(self, input_text, **kwargs):
        try:
            input_text = f"<s>[INST] <<SYS>>{self.system_prompt}<</SYS>>\n{input_text}[/INST]"
            body = json.dumps({"prompt": input_text, "max_gen_len": self.max_gen_len,
                               "temperature": self.temperature, "top_p": self.top_p})
            accept = "application/json"
            contentType = "application/json"

            response = self.bedrock_runtime.invoke_model(
                    body=body, modelId=self.modelId, accept=accept, contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            #print(f"prompt_token_count: {response_body.get('prompt_token_count')}")
            #print(f"generation_token_count: {response_body.get('generation_token_count')}")
            #print(response_body.get("generation"))
            
            return response_body.get("generation")

        except botocore.exceptions.ClientError as error:

            if error.response['Error']['Code'] == 'AccessDeniedException':
                   print(f"\x1b[41m{error.response['Error']['Message']}\
                        \nTo troubeshoot this issue please refer to the following resources.\
                         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

            else:
                raise error

## Load the dataset

Now that we have imported promptbench and set up our Llama2Bedrock class, we will now load and define the dataset that we will use to evaluate our prompt accuracy.

Promptbench supports the following datasets: 'sst2', 'cola', 'qqp', 'mnli', 'mnli_matched', 'qnli', 'wnli', 'rte', 'mrpc', 'mmlu', 'squad_v2', 'un_multi', 'iwslt2017', 'math', 'bool_logic', 'valid_parentheses', 'gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc'.

To load a dataset in promptbench, you can use the "DatasetLoader' interface, which provides a streamlined one-line API for laoding the desired dataset.

In this example we will be using promptbench to load the 'sst2' dataset, which stands for the "Stanford Sentiment Treebank" dataset, which is widely used to support sentiment analysis tasks. Loading a dataset in promptbench is essential for evaluating and understanding large language models (LLMs) and is a fundamental step for leveraging the library's unified framework to analyze the performance of LLMs acorss different tasks and datasets. 

In [124]:
# print all supported datasets in promptbench
print('All supported datasets: ')
print(pb.SUPPORTED_DATASETS)

# load a dataset, sst2, for instance.
# if the dataset is not available locally, it will be downloaded automatically.
dataset_name = "sst2"
dataset = pb.DatasetLoader.load_dataset(dataset_name)

# print the first 5 examples
dataset[:5]

All supported datasets: 
['sst2', 'cola', 'qqp', 'mnli', 'mnli_matched', 'mnli_mismatched', 'qnli', 'wnli', 'rte', 'mrpc', 'mmlu', 'squad_v2', 'un_multi', 'iwslt2017', 'math', 'bool_logic', 'valid_parentheses', 'gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc']


[{'content': "it 's a charming and often affecting journey . ", 'label': 1},
 {'content': 'unflinchingly bleak and desperate ', 'label': 0},
 {'content': 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
  'label': 1},
 {'content': "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
  'label': 1},
 {'content': "it 's slow -- very , very slow . ", 'label': 0}]

***Note: In the cell above, we can see that the 'sst2' dataset has the format of 'content' and 'label', with content being the tokens as a string of sentences and label having a binary classification of 1 and 0.***

## Load the model(s)

Now we can easily load LLM models via the promptbench API.

Loading the model is essential to evaluating and understand the provided LLM, as users can perform various analysis such as prompt engineering, model performance comparison, and robustness testing. 

All supported models include: [['google/flan-t5-large', 'llama2-7b', 'llama2-7b-chat', 'llama2-13b', 'llama2-13b-chat', 'llama2-70b', 'llama2-70b-chat', 'phi-1.5', 'palm', 'gpt-3.5-turbo', 'gpt-4', 'gpt-4-1106-preview', 'gpt-3.5-turbo-1106', 'vicuna-7b', 'vicuna-13b', 'vicuna-13b-v1.3', 'google/flan-ul2']

However, we will be using the llama2-13b-chat model which is available via Amazon Bedrock. Additionally, we will set the model parameters with the Llama2Bedrock class that was just created.

In [132]:
# print all supported models in promptbench
print('All supported models: ')
print(pb.SUPPORTED_MODELS)

# load a model, flan-t5-large, for instance.
modelId = "meta.llama2-70b-chat-v1"
model = Llama2Bedrock(modelId=modelId, bedrock_runtime=bedrock_runtime, max_gen_len=1024, temperature=0.2, top_p=0.9)
# model = pb.LLMModel(model='llama2-13b-chat', max_new_tokens=10, temperature=0.0001)
#help(model)

All supported models: 
['google/flan-t5-large', 'llama2-7b', 'llama2-7b-chat', 'llama2-13b', 'llama2-13b-chat', 'llama2-70b', 'llama2-70b-chat', 'phi-1.5', 'phi-2', 'palm', 'gpt-3.5-turbo', 'gpt-4', 'gpt-4-1106-preview', 'gpt-3.5-turbo-1106', 'vicuna-7b', 'vicuna-13b', 'vicuna-13b-v1.3', 'google/flan-ul2', 'gemini-pro', 'mistralai/Mistral-7B-v0.1', 'mistralai/Mistral-7B-Instruct-v0.1', 'mistralai/Mixtral-8x7B-v0.1', '01-ai/Yi-6B', '01-ai/Yi-34B', '01-ai/Yi-6B-Chat', '01-ai/Yi-34B-Chat', 'baichuan-inc/Baichuan2-7B-Base', 'baichuan-inc/Baichuan2-13B-Base', 'baichuan-inc/Baichuan2-7B-Chat', 'baichuan-inc/Baichuan2-13B-Chat']


## Constructing the prompt

Prompts are the key interaction interface for LLMs. In the following next 2 cells, we will construct sentiment analysis prompts that will feed the content in the 'sst2' dataset that we loaded, and then we will predict against whether or not the sentence was positive: 1 or negative: 0.

We will need to define the projection function for the model output, which is what we will do when running the def proj_func() call. Given the model output will be expecting 'negative' and 'positive', we can map those to binary numbers.

In [133]:
# Prompt API supports a list, so you can pass multiple prompts at once.
prompts = pb.Prompt(["Classify the sentence as positive or negative: {content}",
                     "Determine the emotion of the following sentence as positive or negative: {content}"
                     ])

In [134]:
def proj_func(pred):
    mapping = {
        "positive": 1,
        "negative": 0
    }
    return mapping.get(pred, -1)

## Model evaluation phase

After loading the dataset, llama2-13b model and constructing the array of prompts to be evaluated against the 'sst2' dataset, we can now perform standard evaluation using the loaded prompts and labels.

***Note:*** The predictions may take up to 5 minutes to load given it is iterating over a large number of prompts in the given array. 

In [135]:
from tqdm import tqdm
for prompt in prompts:
    preds = []
    labels = []
    for data in tqdm(dataset):
        # process input
        input_text = pb.InputProcess.basic_format(prompt, data)
        label = data['label']
        raw_pred = model(input_text)
        # process output
        pred = pb.OutputProcess.cls(raw_pred, proj_func)
        preds.append(pred)
        labels.append(label)
    
    # evaluate
    score = pb.Eval.compute_cls_accuracy(preds, labels)
    print(f"{score:.3f}, {prompt}")

100%|██████████| 872/872 [1:05:45<00:00,  4.52s/it]


0.081, Classify the sentence as positive or negative: {content}


100%|██████████| 872/872 [1:04:56<00:00,  4.47s/it]

0.125, Determine the emotion of the following sentence as positive or negative: {content}


### Model Accuracy

Given the output, we can see that the llama2 13b model predicted 18% accuracy on classifying whether or not a sentence is positive or negative, and 0% accuracy when determining whether an emotion is positive or negative using the 'sst2' dataset.

## Adding new modules

As demonstrated, promptbench can be used to evaluate different prompt engineering techniques with different models and datasets. However, it can also be extended to be used to add your own custom datasets, models, prompt engineering methods, and evaluation metrics.

Promptbench can be used for the following use cases as well:
- Create your own custom data set to test and evaluate various models / prompt engineering technniques.
- Using your own fine-tuned model to evaluate against various prompt engineering techniques.
- Add new prompt engineering methods.

#### Adding new datasets involves two steps:

Implementing a New Dataset Class: Datasets are supposed to be implemented in dataload/dataset.py and inherit from the Dataset class. For your custom dataset, implement the __init__ method to load your dataset. We recommend organizing your data samples as dictionaries to facilitate the input process.

Adding an Interface: After customizing the dataset class, register it in the DataLoader class within dataload.py.

#### Similar to adding new datasets, the addition of new models also consists of two steps.

Implementing a New Model Class: Models should be implemented in dataload/model.py, inheriting from the LLMModel class. In your customized model, you should implement self.tokenizer and self.model. You may also customize your own predict function for inference. If the predict function is not customized, the default predict function inherited from LLMModel will be used.

Adding an Interface: After customizing the model class, register it in the _create_model function within the class LLMModel in __init__.py.

### New Dataset

In [ ]:
from datasets import load_dataset
data = load_dataset("narrativeqa", "main")["test"]


In [ ]:
from datasets import load_dataset

class ARC(pb.dataload.Dataset):
    """
    ARC is a dataset class for the AI2 Reasoning Challenge dataset. 
    This dataset is loaded from huggingface datasets: arc (test set).

    Reference:
    https://huggingface.co/datasets/narrativeqa
    AI2 Reasoning Challenge (ARC) (https://arxiv.org/abs/1803.05457)

    Example data format:
    {
        'id': 'Mercury_7175875', 
        'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?', 
        'choices': {'text': ['Planetary density will decrease.', 'Planetary years will become longer.', 'Planetary days will become shorter.', 'Planetary gravity will become stronger.'], 'label': ['A', 'B', 'C', 'D']}, 
        'answerKey': 'C'
    }
    """
    def __init__(self, name):
        data = load_dataset("ai2_arc", name)["test"]
        self.data = []

        for d in data:
            self.data.append(d)

### CyberSecEval

Now that we understand how to evaluate Llama 2 models using promptbench, we will go into Purple Llama's CyberSecEval. 

CyberSecEval is a comprehensive benchmark developed to help bolster the cybersecurity of LLMs used as coding assistants. 

In [86]:
!pip install -r ../PurpleLlama/CybersecurityBenchmarks/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 126.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 139.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.7/100.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 131.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/

## Instructions

Open a terminal and run the following commands:

### Activate Virtual Environment

In [ ]:
%sh
cd ~/SageMaker/PurpleLlama
source ~/.venvs/CybersecurityBenchmarks/bin/activate

### Simplify the following commands by setting a DATASETS environment variable

In [ ]:
%sh
export DATASETS=$PWD/CybersecurityBenchmarks/datasets

### Run benchmarks for Llama-based models on Bedrock

In [ ]:
%sh
python3 -m CybersecurityBenchmarks.benchmark.run \
   --benchmark=instruct \
   --prompt-path="$DATASETS/instruct/instruct.json" \
   --response-path="$DATASETS/instruct_responses.json" \
   --stat-path="$DATASETS/instruct_stat.json" \
   --expansion-llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --judge-llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --llm-under-test="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --run-llm-in-parallel

In [ ]:
python3 -m CybersecurityBenchmarks.benchmark.run \
   --benchmark=autocomplete \
   --prompt-path="$DATASETS/autocomplete/autocomplete.json" \
   --response-path="$DATASETS/autocomplete_responses.json" \
   --stat-path="$DATASETS/instruct_stat.json" \
   --expansion-llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --judge-llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --llm-under-test="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY" \
   --run-llm-in-parallel

### Test: (WORK IN PROGRESS)

In [109]:
run.main(
    benchmark="mitre",
    prompt_path=f"{DATASETS}/mitre/mitre_benchmark_100_per_category_with_augmentation.json",
    response_path=f"{DATASETS}/mitre_responses.json",
    judge_response_path=f"{DATASETS}/mitre_judge_responses.json",
    stat_path=f"{DATASETS}/mitre_stat.json",
    judge_llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY",
    expansion_llm="Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY",
    llms_under_test=["Llama2Bedrock::meta.llama2-13b-chat-v1::EMPTY"],
   )

usage: ipykernel_launcher.py [-h] --benchmark {autocomplete,instruct,mitre}
                             --llm-under-test LLM_UNDER_TEST --prompt-path
                             PROMPT_PATH --response-path RESPONSE_PATH
                             [--stat-path STAT_PATH] [--judge-llm JUDGE_LLM]
                             [--expansion-llm EXPANSION_LLM]
                             [--judge-response-path JUDGE_RESPONSE_PATH]
                             [--run-llm-in-parallel]
                             [--num-queries-per-prompt NUM_QUERIES_PER_PROMPT]
                             [--test] [--debug]
ipykernel_launcher.py: error: the following arguments are required: --benchmark, --llm-under-test, --prompt-path, --response-path


SystemExit: 2